In [ ]:
!ls /data/hub/models--meta-llama--Llama-4-Scout-17B-16E/snapshots/14d516bdff6ac06cec40678529222f193386189c

In [ ]:
# !pip install  torchtune

In [ ]:
# !git clone https://github.com/pytorch/torchtune.git

In [ ]:
cd torchtune/

In [ ]:
!ls recipes/configs/llama4scout_17B_16E_lora.yaml

In [ ]:
# !pip install --pre --upgrade torchtune --extra-index-url https://download.pytorch.org/whl/nightly/cu126

In [ ]:
!cat /opt/conda/lib/python3.12/site-packages/torchtune/training/checkpointing/_utils.py

In [ ]:
!pip install -U wandb

In [ ]:
# !ls /data/hub/models--meta-llama--Llama-4-Scout-17B-16E/snapshots/14d516bdff6ac06cec40678529222f193386189c
# !ls /data/hub/models--meta-llama--Llama-4-Scout-17B-16E-Instruct/snapshots/92f3b1597a195b523d8d9e5700e57e4fbb8f20d3/

In [ ]:
# !tune run --nproc_per_node 8 lora_finetune_distributed --config /home/jovyan/torchtune/recipes/configs/llama4/scout_17B_16E_lora.yaml \
#   checkpointer.checkpoint_dir=/data/hub/models--meta-llama--Llama-4-Scout-17B-16E/snapshots/14d516bdff6ac06cec40678529222f193386189c \
#   tokenizer.path=/data/hub/models--meta-llama--Llama-4-Scout-17B-16E/snapshots/14d516bdff6ac06cec40678529222f193386189c/tokenizer.model \
#   output_dir=/data/output/llama4a \
#     optimizer.lr=1e-6 \
#   clip_grad_norm=1.0 \
#   metric_logger._component_=torchtune.training.metric_logging.WandBLogger \
#   metric_logger.project="llama4_lora" \
#   log_every_n_steps=5


In [ ]:
!tune run --nproc_per_node 8 lora_finetune_distributed --config /home/jovyan/torchtune/recipes/configs/llama4/scout_17B_16E_lora.yaml \
  checkpointer.checkpoint_dir=/data/hub/models--meta-llama--Llama-4-Scout-17B-16E-Instruct/snapshots/92f3b1597a195b523d8d9e5700e57e4fbb8f20d3 \
  tokenizer.path=/data/hub/models--meta-llama--Llama-4-Scout-17B-16E/snapshots/14d516bdff6ac06cec40678529222f193386189c/tokenizer.model \
model.fusion_trainable="frozen" \
model.apply_lora_to_mlp=false \
  output_dir=/data/output/llama4a \
    optimizer.lr=1e-7 \
    optimizer.weight_decay=0.0 \
  clip_grad_norm=0.001 \
  model.lora_attn_modules="""['q_proj',]""" \
  metric_logger._component_=torchtune.training.metric_logging.WandBLogger \
  metric_logger.project="llama4_lora" \
  log_every_n_steps=5


In [ ]:
download_directory = "/data/hub/"

In [ ]:
import json
import logging
import os
from typing import Optional

from huggingface_hub import snapshot_download
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
log = logging.getLogger(__name__)

In [ ]:
def download_model(repo_id: str, max_workers: Optional[int] = 32):
    log.info(f"downloading {repo_id}")
    _id = repo_id.replace(".", "__").replace("-", "_")
    local_dir = os.path.join(download_directory, _id)
    snapshot_download(
        repo_id=repo_id,
        repo_type="model",
        revision=None,
        cache_dir=None,
        max_workers=max_workers,
        ignore_patterns=["*.bin"],
        local_dir=local_dir,
        token=os.environ["HUGGING_FACE_HUB_TOKEN"],
    )
    log.info(f"saved {local_dir}")

In [ ]:
repo_id = "meta-llama/Llama-4-Scout-17B-16E"

In [ ]:
download_model(repo_id)

In [ ]:
# !pip3 install torchtune torchao

In [ ]:
# !tune download meta-llama/Llama-4-Scout-17B-16E

In [ ]:
!mkdir -p /home/jovyan/llama4/scout_17B_16E_lora

In [ ]:
!tune run --nproc_per_node 8 lora_finetune_distributed --config llama4/scout_17B_16E_lora \
  metric_logger._component_=torchtune.utils.metric_logging.WandBLogger \
  metric_logger.project="llama4_lora" \
  log_every_n_steps=5